# Especificação:
## Este é um projeto de DataScience desenvolvido pelo aluno Vitor Alexandrino, para a disciplina CAP394 - Introdução à Data Science, ministrada pelo Professor Rafael Santos no INPE
### Conjunto de dados:
Os dados que aqui serão analisados estão disponíveis para download em https://github.com/vitorAlexandrino/projeto394 sob o nome de: "googleplaystore.csv", e contém um conjunto de informções sobre  cerca de 10 mil aplicativos disponíveis na Google play Store,estas informações estão inicialmente distribuidos em 14 columas, com dados como a categoria que pertence, avaliação média de seus usuários, quantidade de downloads, etc.

### Perguntas que se pretende responder.
Qual o preço que maximiza o lucro de seu aplicativo?

Qual a categoria de aplicativo mais lucrativa?

Quanto o tamanho do aplicativo influencia na sua avaliação?

Quanto o preço do aplicativo influencia na sua avaliação média?

Quanto a avaliação média influencia na quantidade de downloads?

Qual o tamanho ideal para a categoria de um aplicativo?

In [ ]:
#foi necessário instalar os pacotes ploty e cofflinks na base do anaconda

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
plt.style.use('ggplot')

import seaborn as sns
color = sns.color_palette()
sns.set(rc={'figure.figsize':(25,15)})

import plotly
plotly.offline.init_notebook_mode(connected=True)
import plotly.graph_objs as go

import plotly.figure_factory as ff
import cufflinks as cf


import warnings
warnings.filterwarnings('ignore')

## Uma amostra do conjunto de dados

In [3]:
df = pd.read_csv('googleplaystore.csv')

#print(df.dtypes)
#df.loc[df.App=='Tiny Scanner - PDF Scanner App']
# df[df.duplicated(keep='first')]
df.drop_duplicates(subset='App', inplace=True)
df = df[df['Android Ver'] != np.nan]
df = df[df['Android Ver'] != 'NaN']
df = df[df['Installs'] != 'Free']
df = df[df['Installs'] != 'Paid']

#print(len(df))

In [4]:
print('numero de aplicativos no conjunto de dados : ' , len(df))
df.sample(7)

numero de aplicativos no conjunto de dados :  9659


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
6615,Hypertension Hi blood pressure,MEDICAL,3.8,292,2.7M,"100,000+",Free,0,Everyone,Medical,"April 16, 2018",43.0,4.0.3 and up
3691,Samsung Video Library,VIDEO_PLAYERS,4.4,25922,6.1M,"50,000,000+",Free,0,Everyone,Video Players & Editors,"May 9, 2018",1.4.08.2,7.0 and up
102,Mirror - Zoom & Exposure -,BEAUTY,3.9,32090,Varies with device,"1,000,000+",Free,0,Everyone,Beauty,"October 24, 2016",Varies with device,Varies with device
7074,WISE- MOBILE PORTAL,TOOLS,NaN,3,1.0M,500+,Free,0,Everyone,Tools,"July 30, 2015",1.0.0.4,2.2 and up
5092,AG Screen Recorder,TOOLS,3.7,7,2.7M,"1,000+",Free,0,Everyone,Tools,"April 3, 2018",3.0,5.0 and up
6179,Es-Bg Offline Voice Translator,TRAVEL_AND_LOCAL,NaN,1,87M,10+,Paid,$1.49,Everyone,Travel & Local,"March 19, 2017",2.55,3.0 and up
8615,ES Remote,FAMILY,3.3,707,15M,"100,000+",Free,0,Everyone,Entertainment,"November 20, 2014",2.1.1,2.3 and up


In [5]:
#como lidar com o tamnanho "varies with device" ???

Limpando os dados

In [7]:
df['Installs'] = df['Installs'].apply(lambda x: x.replace('+', '') if '+' in str(x) else x)
df['Installs'] = df['Installs'].apply(lambda x: x.replace(',', '') if ',' in str(x) else x)
df['Installs'] = df['Installs'].apply(lambda x: int(x))
#ainda falta tirar $ do preço, ajeitar a data, e decidir como vou lidar com os "NaN" na coluna "Rating", tem aplicativo que tem mais de uma categoria
#por que aparecem linhas em branco???

In [8]:
df.sample(100)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
5599,Star Wars™: Force Arena,FAMILY,4.4,228130,70M,5000000,Free,0,Everyone 10+,Strategy,"July 19, 2018",3.1.4,4.1 and up
6780,ProGrade BT,TOOLS,NaN,0,46M,500,Free,0,Everyone,Tools,"April 27, 2018",1.0.2,4.3 and up
1333,Relax with Andrew Johnson Lite,HEALTH_AND_FITNESS,4.3,2885,Varies with device,100000,Free,0,Everyone,Health & Fitness,"June 19, 2012",Varies with device,Varies with device
2907,EyeEm - Camera & Photo Filter,PHOTOGRAPHY,4.2,215343,44M,10000000,Free,0,Everyone,Photography,"July 14, 2018",6.4.3,4.0.3 and up
2801,TouchNote: Cards & Gifts,PHOTOGRAPHY,4.1,19232,28M,1000000,Free,0,Everyone,Photography,"August 6, 2018",7.4.2,4.4 and up
2073,Super School: Educational Kids Games & Rhymes,FAMILY,4.5,1791,56M,500000,Free,0,Everyone,Education;Education,"June 2, 2018",5.3.11,5.0 and up
1673,PAC-MAN,GAME,4.2,685981,37M,100000000,Free,0,Everyone,Arcade,"May 15, 2018",6.6.3,4.1 and up
8972,US DV Lottery 2019,TRAVEL_AND_LOCAL,4.6,14,4.0M,1000,Free,0,Everyone,Travel & Local,"April 12, 2018",1.0,4.0 and up
5135,Mukwonago AH,MEDICAL,NaN,1,29M,100,Free,0,Everyone,Medical,"June 29, 2017",300000.0.85,4.0.3 and up
9369,Betul Ke Bohong Eh?,GAME,4.2,601,6.4M,100000,Free,0,Everyone,Trivia,"April 3, 2015",1.0.1,2.3 and up
